In [14]:
import os
import numpy as np
import pandas as pd

In [21]:
columns_max = ['train_accuracy', 'test_accuracy', 'roc_auc',
           'precision_0', 'recall_0', 'f1_0', 
           'precision_1', 'recall_1', 'f1_1',
           'ks_stat','tp', 'tn']

# columns_max = ['train_accuracy', 'test_accuracy', 'roc_auc', 
#                'f1_0', 'f1_1', 'ks_stat','tp', 'tn']

# columns_max = ['tp', 'tn']

columns_min = ['fp','fn']
top_n = 1000

columns_name = ['method','k_neighbour', 'train_accuracy', 'test_accuracy', 'roc_auc',
                'precision_0', 'recall_0', 'f1_0', 
                'precision_1', 'recall_1', 'f1_1',
                'ks_stat', 'p_value', 
                'tp', 'tn', 'fp', 'fn']

root_dir = "./class_imbalance_methods_evaluations/"

In [22]:
def best_k_value(file_loc, top_n):
    df = pd.read_csv(file_loc)
    all_col_top_rows = {}
    
    for col in columns_max:
        top_rows = df.sort_values(by=[col],ascending=False)[:top_n]['k_neighbour']
        all_col_top_rows[col + '_top_rows'] = set(top_rows)

    for col in columns_min:
        top_rows = df.sort_values(by=[col],ascending=True)[:top_n]['k_neighbour']
        all_col_top_rows[col + '_top_rows'] = set(top_rows)

    # Finding rows which are common in all the top_rows
    common_rows = set()
    for key,value in all_col_top_rows.items():
        if len(common_rows) == 0:
            common_rows = value
        else:
            common_rows = common_rows.intersection(value)
    #     break
    
    top_rows = df[df['k_neighbour'].isin(common_rows)]
    
#     # Sorting using ks_stat
#     top_ks_stat = df.sort_values(by=['ks_stat'],ascending=False)[:top_n]['k_neighbour']
    
#     # Sorting using fn
#     top_fn = df.sort_values(by=['fn'],ascending=True)[:top_n]['k_neighbour']
    
#     # Sorting using fp
#     top_fp = df.sort_values(by=['fp'],ascending=True)[:top_n]['k_neighbour']
    
#     best_k = set(top_ks_stat).intersection(set(top_fn)).intersection(set(top_fp))
    
#     best_rows = df[df['k_neighbour'].isin(best_k)]
    
    return top_rows.sort_values(by=['fn'],ascending=True)

In [23]:
best_k_value("./class_imbalance_methods_evaluations/app_2_adasyn_eval.csv",1000)

,k_neighbour,train_accuracy,test_accuracy,roc_auc,precision_0,recall_0,f1_0,precision_1,recall_1,f1_1,ks_stat,p_value,tp,tn,fp,fn
1019,1020,0.674011,0.6595,0.675582,0.895147,0.648462,0.752093,0.338061,0.702703,0.456504,0.358090,0.0,286,1033,560,121
1052,1053,0.672149,0.6595,0.675582,0.895147,0.648462,0.752093,0.338061,0.702703,0.456504,0.360547,0.0,286,1033,560,121
186,187,0.671249,0.6640,0.678407,0.895959,0.654112,0.756168,0.341697,0.702703,0.459807,0.372690,0.0,286,1042,551,121
1115,1116,0.672537,0.6585,0.674955,0.894965,0.647207,0.751184,0.337264,0.702703,0.455777,0.357462,0.0,286,1031,562,121
220,221,0.669435,0.6630,0.676865,0.895099,0.653484,0.755443,0.340502,0.700246,0.458199,0.372062,0.0,285,1041,552,122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586,587,0.672071,0.6425,0.658508,0.887125,0.631513,0.737807,0.322171,0.685504,0.438335,0.327114,0.0,279,1006,587,128
1263,1264,0.673856,0.6415,0.657881,0.886926,0.630257,0.736881,0.321429,0.685504,0.437647,0.325231,0.0,279,1004,589,128
1264,1265,0.672847,0.6420,0.658194,0.887026,0.630885,0.737344,0.321799,0.685504,0.437991,0.325339,0.0,279,1005,588,128
604,605,0.675562,0.6435,0.659136,0.887324,0.632768,0.738732,0.322917,0.685504,0.439024,0.326487,0.0,279,1008,585,128


In [24]:
df = pd.DataFrame([], columns=columns_name)
best_rows_values = {}
for file in sorted(os.listdir(root_dir)):
    
    # Change value of file.split('_')[1] based on approach  i.e. [1,2]
    if file[-3:] == 'csv' and file.split('_')[1] == '2':
        approach = file.split('_')[1]
        method_name = file.split('_')[2].upper()
        
        best_rows = best_k_value(os.path.join(root_dir, file),top_n)
        print(file, len(best_rows))
        
        best_rows_values[method_name] = list(best_rows['k_neighbour'].values)
        
        best_rows.insert(0,'method' ,[f'Approach_{approach}_{method_name}']* len(best_rows) )
        
        df = pd.concat([df, best_rows[:1]])

    #     break

app_2_adasyn_eval.csv 368
app_2_borderline_smote_eval.csv 393
app_2_kmeans_smote_eval.csv 338
app_2_smote_eval.csv 283
app_2_smoteenn_eval.csv 234
app_2_smoten_eval.csv 62
app_2_smotenc_eval.csv 149
app_2_smotetomek_eval.csv 475
app_2_svmsmote_eval.csv 545


In [15]:
np.save('./final_results/class_imbalance_methods_evaluations/best_neighbour_values.npy', best_rows_values)

{'ADASYN': [1020,
  1053,
  187,
  1116,
  221,
  186,
  1025,
  1215,
  905,
  1093,
  841,
  1040,
  1170,
  662,
  729,
  730,
  1112,
  737,
  739,
  1342,
  757,
  768,
  799,
  1097,
  1148,
  1028,
  874,
  952,
  1343,
  115,
  881,
  888,
  894,
  1567,
  943,
  944,
  164,
  1044,
  191,
  1336,
  532,
  1197,
  1229,
  551,
  607,
  614,
  1304,
  1306,
  664,
  426,
  580,
  1005,
  1156,
  1473,
  711,
  895,
  1444,
  1086,
  1417,
  809,
  1159,
  586,
  1252,
  1573,
  1206,
  628,
  673,
  1176,
  1133,
  1175,
  1174,
  637,
  1173,
  651,
  654,
  1153,
  658,
  684,
  674,
  630,
  1271,
  16,
  1258,
  1279,
  1337,
  401,
  1305,
  393,
  354,
  1467,
  1289,
  392,
  1285,
  569,
  78,
  1284,
  1409,
  1402,
  1398,
  1397,
  1391,
  1382,
  1380,
  1124,
  1129,
  1550,
  1379,
  1371,
  1370,
  1369,
  1365,
  1107,
  1106,
  1058,
  1414,
  1525,
  1499,
  1471,
  1082,
  1528,
  1536,
  1460,
  1545,
  1437,
  1436,
  1435,
  1430,
  1426,
  1416,
  1364,
  

In [5]:
# Sorting using ks_stat
top_ks_stat = df.sort_values(by=['fp'],ascending=True)[:top_n]['k_neighbour']

# Sorting using ks_stat
top_fn = df.sort_values(by=['fn'],ascending=True)[:top_n]['k_neighbour']

best_k = set(top_ks_stat).intersection(set(top_fn))

df = df[df['k_neighbour'].isin(best_k)]

df.sort_values(by=['ks_stat'],ascending=False, inplace=True)

In [7]:
df.to_csv('./top_class_balancing_methods.csv', header=True, index=False)

In [15]:
df

,method,k_neighbour,train_accuracy,test_accuracy,roc_auc,precision_0,recall_0,f1_0,precision_1,recall_1,f1_1,ks_stat,p_value,tp,tn,fp,fn
1019,Approach_2_ADASYN,1020,0.674011,0.6595,0.675582,0.895147,0.648462,0.752093,0.338061,0.702703,0.456504,0.358090,0.00000,286,1033,560,121
1408,Approach_2_BORDERLINE,1409,0.671115,0.6675,0.676946,0.893888,0.661017,0.760014,0.343066,0.692875,0.458910,0.358360,0.00000,282,1053,540,125
109,Approach_2_KMEANS,110,0.774168,0.6565,0.574919,0.832111,0.712492,0.767670,0.279874,0.437346,0.341323,0.171993,0.00000,178,1135,458,229
272,Approach_2_SMOTE,273,0.672684,0.6680,0.678174,0.894647,0.661017,0.760289,0.343864,0.695332,0.460163,0.371381,0.00000,283,1053,540,124
0,Approach_2_SMOTEENN,1,0.706708,0.5345,0.629126,0.896882,0.469554,0.616399,0.275300,0.788698,0.408137,0.305757,0.00000,321,748,845,86
281,Approach_2_SMOTEN,282,0.797724,0.7020,0.563238,0.823072,0.797238,0.809949,0.293217,0.329238,0.310185,0.128359,0.00004,134,1270,323,273
112,Approach_2_SMOTENC,113,0.668838,0.6710,0.679143,0.894515,0.665411,0.763139,0.346012,0.692875,0.461538,0.368350,0.00000,282,1060,533,125
115,Approach_2_SMOTETOMEK,116,0.667769,0.6605,0.676210,0.895329,0.649718,0.753001,0.338863,0.702703,0.457234,0.357408,0.00000,286,1035,558,121
147,Approach_2_SVMSMOTE,148,0.630691,0.6070,0.627992,0.873265,0.592593,0.706058,0.293798,0.663391,0.407240,0.261997,0.00000,270,944,649,137


In [186]:
methods_ = ['smote', 'adasyn', 'borderline_smote', 'kmeans_smote', 'smoten' , 'svmsmote', 'smoteenn', 'smotetomek']
k_values = [273, 1020, 1409, 110, 282, 148, 1, 116]

2000

In [9]:
df

,method,k_neighbour,train_accuracy,test_accuracy,roc_auc,precision_0,recall_0,f1_0,precision_1,recall_1,f1_1,ks_stat,p_value,tp,tn,fp,fn
1234,Approach_1_SMOTEN,1235,0.858477,0.857815,0.857815,0.830243,0.899561,0.863513,0.890411,0.816070,0.851621,0.744507,0.000000,1300,1433,160,293
1139,Approach_1_SMOTEN,1140,0.858713,0.858757,0.858757,0.830921,0.900816,0.864458,0.891707,0.816698,0.852556,0.743879,0.000000,1301,1435,158,292
1133,Approach_1_SMOTEN,1134,0.858163,0.859071,0.859071,0.830636,0.902072,0.864881,0.892857,0.816070,0.852739,0.743252,0.000000,1300,1437,156,293
476,Approach_1_KMEANS,477,0.772998,0.780603,0.780603,0.807428,0.736974,0.770594,0.758083,0.824231,0.789774,0.571249,0.000000,1313,1174,419,280
460,Approach_1_KMEANS,461,0.774882,0.781230,0.781230,0.808116,0.737602,0.771250,0.758661,0.824859,0.790376,0.568738,0.000000,1314,1175,418,279
442,Approach_1_KMEANS,443,0.776060,0.778092,0.778092,0.806787,0.731325,0.767204,0.754305,0.824859,0.788006,0.561833,0.000000,1314,1165,428,279
147,Approach_1_SMOTEENN,148,0.749604,0.772512,0.766441,0.833333,0.650000,0.730337,0.736842,0.882883,0.803279,0.559520,0.000000,294,195,105,39
110,Approach_1_SMOTEENN,111,0.726061,0.770598,0.762494,0.823009,0.645833,0.723735,0.740458,0.879154,0.803867,0.557412,0.000000,291,186,102,40
216,Approach_1_SMOTEENN,217,0.728477,0.764901,0.760180,0.820961,0.650519,0.725869,0.730667,0.869841,0.794203,0.549756,0.000000,274,188,101,41
271,Approach_1_SMOTETOMEK,272,0.748938,0.752336,0.752336,0.770492,0.718777,0.743736,0.736465,0.785896,0.760378,0.522515,0.000000,925,846,331,252
